In [1]:
from icecube import dataio, icetray , dataclasses 


In [2]:
import pandas as pd
pd.set_option("display.max_columns", None)   


In [3]:
from collections.abc import Iterable

In [4]:
GCD_PATH = "/project/6008051/pone_simulation/GCD_Library/PONE_800mGrid.i3.gz"
data_file = dataio.I3File(GCD_PATH)
data_file.pop_frame()
calibration = data_file.pop_frame()

In [5]:
# dir(calibration)

In [6]:
print(type(calibration))
print("Stop:", calibration.Stop)   
print("Key's:", list(calibration.keys()))


<class 'icecube._icetray.I3Frame'>
Stop: Calibration
Key's: ['I3Calibration', 'I3ModuleGeoMap', 'Subdetectors', 'I3Geometry', 'EndTime', 'I3OMGeoMap', 'StartTime', 'SPEAbove', 'SPEScalingFactors']


In [7]:
# Small helper: print the type and a short summary for each key in the frame
for key in calibration.keys():
    obj = calibration[key]
    print("="*60)
    print(f"Key: {key}")
    print("Type:", type(obj))
    try:
        print("dir(obj) first 10:", dir(obj))
    except Exception as e:
        print("dir(obj) could not be obtained:", e)

Key: I3Calibration
Type: <class 'icecube._dataclasses.I3Calibration'>
dir(obj) first 10: ['__class__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__getstate_manages_dict__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__instance_size__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__safe_for_unpickling__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'dom_cal', 'end_time', 'from_frame', 'start_time', 'vem_cal']
Key: I3ModuleGeoMap
Type: <class 'icecube._dataclasses.I3ModuleGeoMap'>
dir(obj) first 10: ['__class__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__getstate_manages_dict__', '__gt__', '__hash__', '__init__', '__ini

WARN (dataclasses): Reading version 0 from file into version 1 of I3ModuleGeo class, all modules are assumed spherical. (I3ModuleGeo.cxx:75 in void I3ModuleGeo::serialize(Archive&, unsigned int) [with Archive = icecube::archive::portable_binary_iarchive])
WARN (dataclasses): Reading version 0 from file into version 1 of I3ModuleGeo class, all modules are assumed spherical. (I3ModuleGeo.cxx:75 in void I3ModuleGeo::serialize(Archive&, unsigned int) [with Archive = icecube::archive::portable_binary_iarchive])
WARN (dataclasses): Reading version 0 from file into version 1 of I3ModuleGeo class, all modules are assumed spherical. (I3ModuleGeo.cxx:75 in void I3ModuleGeo::serialize(Archive&, unsigned int) [with Archive = icecube::archive::portable_binary_iarchive])
WARN (dataclasses): Reading version 0 from file into version 1 of I3ModuleGeo class, all modules are assumed spherical. (I3ModuleGeo.cxx:75 in void I3ModuleGeo::serialize(Archive&, unsigned int) [with Archive = icecube::archive::por

# I3OMGeoMap (same as Geometry.I3OMGeoMap)

In [8]:
# om_map = calibration["I3OMGeoMap"]
# print(type(om_map))
# print("Number of OMs:", len(om_map))
# 
# # 340 * 20 * 16

In [9]:
# sample_key, sample_geo = next(iter(om_map.items()))
# attrs = [a for a in dir(sample_geo) if not a.startswith("_")]
# 
# # we handle these manually 
# skip = {"position", "orientation", "omtype", "area"}
# attrs = [a for a in attrs if a not in skip]
# 
# rows = []
# 
# for omkey, omgeo in om_map.items():
#     pos = omgeo.position
#     ori = omgeo.orientation
# 
#     row = {
#         "string": omkey.string,
#         "om": omkey.om,
#         "pmt": getattr(omkey, "pmt", None),
# 
#         "x": pos.x,
#         "y": pos.y,
#         "z": pos.z,
# 
#         "dir_x": ori.dir.x,
#         "dir_y": ori.dir.y,
#         "dir_z": ori.dir.z,
# 
#         "up_x": ori.up.x,
#         "up_y": ori.up.y,
#         "up_z": ori.up.z,
# 
#         "right_x": ori.right.x,
#         "right_y": ori.right.y,
#         "right_z": ori.right.z,
# 
#         "omtype": int(omgeo.omtype),
#         "area": float(omgeo.area),
#     }
# 
#     # add all other simple attributes from omgeo
#     for name in attrs:
#         v = getattr(omgeo, name)
#         if isinstance(v, (int, float, str, bool)):
#             row[name] = v
# 
#     rows.append(row)
# 
# om_df = pd.DataFrame(rows)
# om_df.head()
# 

In [10]:
# len(om_df)

In [11]:
# # Columns that have the same value in all rows
# const_cols = om_df.columns[om_df.nunique(dropna=False) == 1]
# print(const_cols)
# 

In [12]:
# # those same values:
# print(om_df[const_cols].iloc[0])
# 

In [13]:
# om_df = om_df.loc[:, om_df.nunique(dropna=False) > 1]


In [14]:
# om_df

# - I3ModuleGeoMap (same as Geometry.I3ModuleGeoMap)


In [15]:
# mod_map = calibration["I3ModuleGeoMap"]
# print(type(mod_map))
# print("Number of modules:", len(mod_map))
# 
# # 340 * 20

In [16]:
# print(mod_map)

In [17]:
# # discover attrs once
# k0, g0 = next(iter(mod_map.items()))
# key_attrs = [a for a in dir(k0) if not a.startswith("_")]
# geo_attrs = [a for a in dir(g0) if not a.startswith("_")]
# 
# rows = []
# 
# for mkey, mgeo in mod_map.items():
#     # try both 'pos' and 'position'
#     pos = getattr(mgeo, "pos", None)
#     if pos is None:
#         pos = getattr(mgeo, "position", None)
# 
#     ori = getattr(mgeo, "orientation", None)
#     d   = getattr(mgeo, "dir", None)
# 
#     row = {}
# 
#     # ---- ModuleKey info ----
#     for a in key_attrs:
#         v = getattr(mkey, a)
#         if isinstance(v, (int, float, str, bool)):
#             row[f"key_{a}"] = v
# 
#     # ---- position ----
#     if pos is not None:
#         row["x"] = pos.x
#         row["y"] = pos.y
#         row["z"] = pos.z
# 
#     # ---- direction vector ----
#     if d is not None:
#         row["dir_x"] = d.x
#         row["dir_y"] = d.y
#         row["dir_z"] = d.z
#         # if zenith/azimuth exist:
#         for name in ("zenith", "azimuth"):
#             if hasattr(d, name):
#                 row[f"dir_{name}"] = getattr(d, name)
# 
#     # ---- orientation vectors ----
#     if ori is not None:
#         for base, vec in (("up", ori.up), ("right", ori.right)):
#             row[f"{base}_x"] = vec.x
#             row[f"{base}_y"] = vec.y
#             row[f"{base}_z"] = vec.z
# 
#     # ---- all other scalar attrs of mgeo ----
#     for a in geo_attrs:
#         if a in {"pos", "position", "orientation", "dir"}:
#             continue
#         if a in row:  # don't overwrite existing keys
#             continue
#         v = getattr(mgeo, a)
#         if isinstance(v, (int, float, str, bool)):
#             row[a] = v
# 
#     rows.append(row)
# 
# mod_df = pd.DataFrame(rows)
# mod_df.head()
# 

In [18]:
# mod_df.shape        


In [19]:
# unique_counts = mod_df.nunique(dropna=False)
# 
# constant_cols = unique_counts[unique_counts == 1].index.tolist()
# varying_cols  = unique_counts[unique_counts > 1].index.tolist()
# 
# print("Columns that are constant across all rows:")
# for c in constant_cols:
#     print("  -", c)
# 
# print("\nColumns that vary:")
# for c in varying_cols:
#     print("  -", c)
# 

In [20]:
# mod_df = mod_df.loc[:, mod_df.nunique(dropna=False) > 1]


In [21]:
# mod_df

# - Subdetectors (same as Geometry.Subdetectors)


In [22]:
# subdets = calibration["Subdetectors"]
# print(type(subdets))

In [23]:
# sample_key = next(iter(subdets.keys()))
# attrs = [a for a in dir(sample_key) if not a.startswith("_")]
# 
# rows = []
# 
# for mkey in subdets.keys():
#     name = subdets[mkey]   
# 
#     row = {"subdetector_name": str(name)}
# 
#     # add simple attributes from ModuleKey
#     for attr in attrs:
#         v = getattr(mkey, attr)
#         if isinstance(v, (int, float, str, bool)):
#             row[attr] = v
# 
#     rows.append(row)
# 
# subdet_df = pd.DataFrame(rows)
# subdet_df
# 

In [24]:
# len(subdets)
# # number of OM

# - I3Geometry (same as Geometry.I3Geometry)

In [25]:
# geo = calibration["I3Geometry"]
# 
# print("Type:", type(geo))
# print("Attributes:")
# print(dir(geo))


In [26]:
# print(geo)

In [27]:
# # All attributes
# all_attrs = dir(geo)
# 
# # Non-dunder attributes
# public_attrs = [a for a in all_attrs if not a.startswith("_")]
# 
# print("Public attrs:")
# for a in public_attrs:
#     print("  -", a)
# 

In [28]:
# rows = []
# for a in public_attrs:
#     value = getattr(geo, a)
#     # try to get length if it behaves like a container
#     try:
#         n = len(value)
#     except TypeError:
#         n = None
# 
#     rows.append({
#         "attr": a,
#         "type": type(value).__name__,
#         "len": n,
#         "repr_sample": repr(value),  
#     })
# 
# geo_attrs_df = pd.DataFrame(rows)
# geo_attrs_df


In [29]:
# for name in public_attrs:
#     val = getattr(geo, name)
# 
#     print(f"\n{name}")
#     print("  type:", type(val))
# 
#     # try to show some size info
#     if hasattr(val, "keys"):
#         print("  n_keys:", len(val))
#     elif hasattr(val, "__len__") and not isinstance(val, (str, bytes)):
#         print("  len:", len(val))
# 
#     # small repr preview
#     print("  repr:", repr(val)[:200])
# 

In [30]:
# om_map  = calibration["I3OMGeoMap"]
# geo     = calibration["I3Geometry"]
# om_map2 = geo.omgeo
# 
# print(len(om_map), len(om_map2))
# 
# print("same keys? :", set(om_map.keys()) == set(om_map2.keys()))
# 
# keys = list(om_map.keys())
# print("all values equal? :",
#       all(om_map[k] == om_map2[k] for k in keys))
# 

In [31]:
# print(om_map is om_map2)


# - StartTime and EndTime (same as Geometry.StartTime and Geometry.EndTime)


In [32]:
# start = calibration["StartTime"]
# end   = calibration["EndTime"]
# print(start)
# print(end)

In [33]:
# start

In [34]:
# end

In [35]:
# print(repr(start))
# print(str(start))
# 

In [36]:
# print(repr(end))
# print(str(end))
# 

# - I3Calibration

In [37]:
# 1. get I3Calibration object
cal_obj = calibration["I3Calibration"]

print("Type:", type(cal_obj))

# list public attributes on the calibration object itself
cal_attrs = [a for a in dir(cal_obj) if not a.startswith("_")]
print("\nCalibration public attrs:")
for a in cal_attrs:
    print(" -", a)


Type: <class 'icecube._dataclasses.I3Calibration'>

Calibration public attrs:
 - dom_cal
 - end_time
 - from_frame
 - start_time
 - vem_cal


## dom_cal

In [38]:
dom_cal = cal_obj.dom_cal
print("dom_cal type & len:", type(dom_cal), len(dom_cal))

# show a few keys
for k in list(dom_cal.keys())[:5]:
    print("key:", k)

# pick one example DOMCalibration
sample_key = next(iter(dom_cal.keys()))
sample_cal = dom_cal[sample_key]
print("\nSample key:", sample_key)
print("Sample cal type:", type(sample_cal))


dom_cal type & len: <class 'icecube._dataclasses.Map_OMKey_I3DOMCalibration'> 108800
key: OMKey(1,1,1)
key: OMKey(1,1,2)
key: OMKey(1,1,3)
key: OMKey(1,1,4)
key: OMKey(1,1,5)

Sample key: OMKey(1,1,1)
Sample cal type: <class 'icecube._dataclasses.I3DOMCalibration'>


In [39]:
dom_attrs = [a for a in dir(sample_cal) if not a.startswith("_")]

rows = []
for name in dom_attrs:
    v = getattr(sample_cal, name)
    rows.append({
        "attr": name,
        "type": type(v).__name__,
        "repr_sample": repr(v)[:80],  # first 80 chars, just for preview
    })

cal_attr_df = pd.DataFrame(rows)
cal_attr_df


,attr,type,repr_sample
0,NEW_TOROID,ToroidType,icecube._dataclasses.ToroidType.NEW_TOROID
1,OLD_TOROID,ToroidType,icecube._dataclasses.ToroidType.OLD_TOROID
2,ToroidType,type,<class 'icecube._dataclasses.ToroidType'>
3,atwd_beacon_baseline,_atwd_beacon_baseline_proxy,<icecube._dataclasses._atwd_beacon_baseline_pr...
4,atwd_bin_calib_slope,_atwd_bin_calib_slope_proxy,<icecube._dataclasses._atwd_bin_calib_slope_pr...
5,atwd_delta_t,_atwd_delta_t_proxy,<icecube._dataclasses._atwd_delta_t_proxy obje...
6,atwd_freq_fit,_atwd_freq_fit_proxy,<icecube._dataclasses._atwd_freq_fit_proxy obj...
7,atwd_gain,_atwd_delta_t_proxy,<icecube._dataclasses._atwd_delta_t_proxy obje...
8,atwd_pulse_template,method,<bound method atwd_pulse_template of <icecube....
9,combined_spe_charge_distribution,SPEChargeDistribution,<icecube._dataclasses.SPEChargeDistribution ob...


In [40]:
# full DOM calibration table using (almost) all attributes

dom_cal = cal_obj.dom_cal

# use one example calibration object to get attribute names
sample_key = next(iter(dom_cal.keys()))
sample_cal = dom_cal[sample_key]
dom_attrs = [a for a in dir(sample_cal) if not a.startswith("_")]

rows = []

for omkey, cal in dom_cal.items():
    row = {
        "string": omkey.string,
        "om": omkey.om,
        "pmt": getattr(omkey, "pmt", None),
    }

    for name in dom_attrs:
        v = getattr(cal, name)

        # 1) methods / functions are not real data → skip
        if callable(v):
            continue

        # 2) simple scalars: keep as is
        if isinstance(v, (int, float, bool, str)):
            row[name] = v
        else:
            # 3) more complex objects: store repr so we don't lose info
            row[name] = repr(v)

    rows.append(row)

dom_cal_df = pd.DataFrame(rows)
dom_cal_df.head()


,string,om,pmt,NEW_TOROID,OLD_TOROID,atwd_beacon_baseline,atwd_bin_calib_slope,atwd_delta_t,atwd_freq_fit,atwd_gain,dom_cal_time,dom_cal_version,dom_noise_decay_rate,dom_noise_rate,dom_noise_scintillation_hits,dom_noise_scintillation_mean,dom_noise_scintillation_sigma,dom_noise_thermal_rate,fadc_baseline_fit,fadc_beacon_baseline,fadc_delta_t,fadc_gain,front_end_impedance,hv_gain_fit,is_mean_atwd_charge_correction_valid,is_mean_fadc_charge_correction_valid,mean_atwd_charge_correction,mean_fadc_charge_correction,mpe_disc_calib,pmt_disc_calib,relative_dom_eff,spe_disc_calib,tau_parameters,temperature,toroid_type,transit_time
0,1,1,1,1,0,<icecube._dataclasses._atwd_beacon_baseline_pr...,<icecube._dataclasses._atwd_bin_calib_slope_pr...,<icecube._dataclasses._atwd_delta_t_proxy obje...,<icecube._dataclasses._atwd_freq_fit_proxy obj...,<icecube._dataclasses._atwd_delta_t_proxy obje...,"I3Time(1858,276480000000000000L)",7.6.0,7.747244e-08,5.877200e-07,10.557505,3.826594,1.702725,2.200675e-07,<icecube._dataclasses.LinearFit object at 0x14...,133.203693,-113.626395,8.853992e-14,7.766126e-18,<icecube._dataclasses.LinearFit object at 0x14...,False,False,NaN,NaN,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,1.0,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.TauParam object at 0x147...,275.0,1,<icecube._dataclasses.LinearFit object at 0x14...
1,1,1,2,1,0,<icecube._dataclasses._atwd_beacon_baseline_pr...,<icecube._dataclasses._atwd_bin_calib_slope_pr...,<icecube._dataclasses._atwd_delta_t_proxy obje...,<icecube._dataclasses._atwd_freq_fit_proxy obj...,<icecube._dataclasses._atwd_delta_t_proxy obje...,"I3Time(1858,276480000000000000L)",7.6.0,7.747244e-08,5.877200e-07,10.557505,3.826594,1.702725,2.200675e-07,<icecube._dataclasses.LinearFit object at 0x14...,133.203693,-113.626395,8.853992e-14,7.766126e-18,<icecube._dataclasses.LinearFit object at 0x14...,False,False,NaN,NaN,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,1.0,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.TauParam object at 0x147...,275.0,1,<icecube._dataclasses.LinearFit object at 0x14...
2,1,1,3,1,0,<icecube._dataclasses._atwd_beacon_baseline_pr...,<icecube._dataclasses._atwd_bin_calib_slope_pr...,<icecube._dataclasses._atwd_delta_t_proxy obje...,<icecube._dataclasses._atwd_freq_fit_proxy obj...,<icecube._dataclasses._atwd_delta_t_proxy obje...,"I3Time(1858,276480000000000000L)",7.6.0,7.747244e-08,5.877200e-07,10.557505,3.826594,1.702725,2.200675e-07,<icecube._dataclasses.LinearFit object at 0x14...,133.203693,-113.626395,8.853992e-14,7.766126e-18,<icecube._dataclasses.LinearFit object at 0x14...,False,False,NaN,NaN,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,1.0,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.TauParam object at 0x147...,275.0,1,<icecube._dataclasses.LinearFit object at 0x14...
3,1,1,4,1,0,<icecube._dataclasses._atwd_beacon_baseline_pr...,<icecube._dataclasses._atwd_bin_calib_slope_pr...,<icecube._dataclasses._atwd_delta_t_proxy obje...,<icecube._dataclasses._atwd_freq_fit_proxy obj...,<icecube._dataclasses._atwd_delta_t_proxy obje...,"I3Time(1858,276480000000000000L)",7.6.0,7.747244e-08,5.877200e-07,10.557505,3.826594,1.702725,2.200675e-07,<icecube._dataclasses.LinearFit object at 0x14...,133.203693,-113.626395,8.853992e-14,7.766126e-18,<icecube._dataclasses.LinearFit object at 0x14...,False,False,NaN,NaN,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,1.0,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.TauParam object at 0x147...,275.0,1,<icecube._dataclasses.LinearFit object at 0x14...
4,1,1,5,1,0,<icecube._dataclasses._atwd_beacon_baseline_pr...,<icecube._dataclasses._atwd_bin_calib_slope_pr...,<icecube._dataclasses._atwd_delta_t_proxy obje...,<ice

In [41]:
dom_cal_df.shape

(108800, 36)

In [42]:
# columns that are identical for all DOMs
const_cols = dom_cal_df.columns[dom_cal_df.nunique(dropna=False) == 1]
print("Constant columns:", list(const_cols))


Constant columns: ['NEW_TOROID', 'OLD_TOROID', 'atwd_beacon_baseline', 'atwd_bin_calib_slope', 'atwd_delta_t', 'atwd_freq_fit', 'atwd_gain', 'dom_cal_time', 'dom_cal_version', 'dom_noise_decay_rate', 'dom_noise_rate', 'dom_noise_scintillation_hits', 'dom_noise_scintillation_mean', 'dom_noise_scintillation_sigma', 'dom_noise_thermal_rate', 'fadc_beacon_baseline', 'fadc_delta_t', 'fadc_gain', 'front_end_impedance', 'is_mean_atwd_charge_correction_valid', 'is_mean_fadc_charge_correction_valid', 'mean_atwd_charge_correction', 'mean_fadc_charge_correction', 'relative_dom_eff', 'temperature', 'toroid_type']


In [43]:

dom_cal_df = dom_cal_df.drop(columns=const_cols)
dom_cal_df

,string,om,pmt,fadc_baseline_fit,hv_gain_fit,mpe_disc_calib,pmt_disc_calib,spe_disc_calib,tau_parameters,transit_time
0,1,1,1,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.TauParam object at 0x147...,<icecube._dataclasses.LinearFit object at 0x14...
1,1,1,2,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.TauParam object at 0x147...,<icecube._dataclasses.LinearFit object at 0x14...
2,1,1,3,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.TauParam object at 0x147...,<icecube._dataclasses.LinearFit object at 0x14...
3,1,1,4,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.TauParam object at 0x147...,<icecube._dataclasses.LinearFit object at 0x14...
4,1,1,5,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.TauParam object at 0x147...,<icecube._dataclasses.LinearFit object at 0x14...
...,...,...,...,...,...,...,...,...,...,...
108795,340,20,12,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.TauParam object at 0x147...,<icecube._dataclasses.LinearFit object at 0x14...
108796,340,20,13,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.TauParam object at 0x147...,<icecube._dataclasses.LinearFit object at 0x14...
108797,340,20,14,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.TauParam object at 0x147...,<icecube._dataclasses.LinearFit object at 0x14...
108798,340,20,15,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.LinearFit object at 0x14...,<icecube._dataclasses.TauParam object at 0x147...,<icecube._dataclasses.LinearFit object at 0x14...


In [44]:
## bu kismi dusun biraz. bunlar lazim mi??

## vem_cal

In [45]:
print("vem_cal len:", len(cal_obj.vem_cal))


vem_cal len: 0


# SPEAbove
##### same for all OMs??

In [46]:
spe_above = calibration["SPEAbove"]

print("Type:", type(spe_above))

Type: <class 'icecube._dataclasses.I3MapKeyDouble'>


In [47]:
try:
    print("Number of entries:", len(spe_above))
except TypeError:
    print("len() not defined")

# peek at a few entries
print("\nFirst few entries:")
for i, (k, v) in enumerate(spe_above.items()):
    print("  key:", k, "   value:", v)

Number of entries: 1

First few entries:
  key: OMKey(0,0,0)    value: 0.6766608690779268


In [48]:
print(spe_above)

[OMKey(0,0,0) => 0.676661]


In [49]:
if len(spe_above) == 1:
    (only_key, only_val), = spe_above.items()
    print("Single entry in SPEAbove:")
    print("  key  :", only_key)
    print("  value:", only_val)
else:
    print("SPEAbove has", len(spe_above), "entries.")


spe_above_df = pd.DataFrame(
    [{"string": only_key.string,
      "om": only_key.om,
      "pmt": getattr(only_key, "pmt", None),
      "value": only_val}]
)
spe_above_df


Single entry in SPEAbove:
  key  : OMKey(0,0,0)
  value: 0.6766608690779268


,string,om,pmt,value
0,0,0,0,0.676661


# - SPEScalingFactors

In [50]:
spe_scale = calibration["SPEScalingFactors"]

print("Type:", type(spe_scale))

Type: <class 'icecube._dataclasses.I3MapKeyDouble'>


In [51]:
try:
    print("Number of entries:", len(spe_scale))
except TypeError:
    print("len() not defined")

print("\nFirst few entries:")
for i, (k, v) in enumerate(spe_scale.items()):
    print("  key:", k, "   value:", v)


Number of entries: 1

First few entries:
  key: OMKey(0,0,0)    value: 0.768802633506287


In [52]:
rows = []
for k, v in spe_scale.items():
    rows.append({
        "string": getattr(k, "string", None),
        "om": getattr(k, "om", None),
        "pmt": getattr(k, "pmt", None),
        "key_repr": str(k),
        "value": v,
    })

spe_scale_df = pd.DataFrame(rows)
spe_scale_df.head()

,string,om,pmt,key_repr,value
0,0,0,0,"OMKey(0,0,0)",0.768803
